## Extraction from Coin Gecko's API 
## Now lets turn it all into functions and adjust it to work with prefect workflow 

In [1]:
import requests
def extract():
    #Set the base URL for CoinGecko API
    base_url = "https://api.coingecko.com/api/v3"

    #Specify the endpoint for fetching trending coins
    trennding_coins_endpoint = "/coins/markets"

    #Paremeters for API request

    params_trending_coins = {
        'vs_currency': 'usd',
        'order': 'volume_desc',
        'per_page': 10,
        'page': 1
    }

    #Send get request to api endpoint with specified parameters
    response = requests.get(base_url + trennding_coins_endpoint, params=params_trending_coins)

    #parse the json response to a python dictionary
    data_trending_coins = response.json()

    return data_trending_coins

[{'id': 'tether',
  'symbol': 'usdt',
  'name': 'Tether',
  'image': 'https://assets.coingecko.com/coins/images/325/large/Tether.png?1696501661',
  'current_price': 0.999538,
  'market_cap': 94854054281,
  'market_cap_rank': 3,
  'fully_diluted_valuation': 94854054281,
  'total_volume': 18152024243,
  'high_24h': 1.001,
  'low_24h': 0.99605,
  'price_change_24h': 0.00053645,
  'price_change_percentage_24h': 0.0537,
  'market_cap_change_24h': -23043380.0443573,
  'market_cap_change_percentage_24h': -0.02429,
  'circulating_supply': 94930821669.2984,
  'total_supply': 94930821669.2984,
  'max_supply': None,
  'ath': 1.32,
  'ath_change_percentage': -24.47749,
  'ath_date': '2018-07-24T00:00:00.000Z',
  'atl': 0.572521,
  'atl_change_percentage': 74.53243,
  'atl_date': '2015-03-02T00:00:00.000Z',
  'roi': None,
  'last_updated': '2024-01-22T01:50:12.530Z'},
 {'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': 'https://assets.coingecko.com/coins/images/1/large/bitcoin.png

## Transformation 

In [33]:
import pandas as pd
from forex_python.converter import CurrencyRates


def transform(data_trending_coins)
    df = pd.DataFrame(data_trending_coins)

    #select the first 9 columns (index 0 to 8)
    df =  df.iloc[:, :9]
    # drop image column
    df = df.drop('image' , axis=1)

    #define the function that converts prices
    def convert_price(row, to_currency):
        c = CurrencyRates()

        #get current price from dataframe row
        price = row['current_price']
        #since our currency is in USD
        from_currency = 'USD'

        #FETCHING the exchange rates from CurrencyRates object
        exchange_rate = c.get_rate(from_currency,to_currency)

        #calculate the price
        converted_price = price * exchange_rate

        return converted_price

    #list of target currencies for conversion
    currencies = ['EUR', 'GBP', 'JPY']

    #iterate throught each target currency
    for currency in currencies:
        #apply the convert_price function to each row, creating new columns in the df dataframe
        df[f'price_in_{currency}'] = df.apply(lambda row: convert_price(row, currency),axis = 1)

    # we need a date column, we'll delete the former table in our database 
    today = pd.to_datetime('today')
    df[today] = today
    return df

,id,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,price_in_EUR,price_in_GBP,price_in_JPY,2024-01-22 02:23:01.592837
0,tether,usdt,Tether,0.999538,94854054281,3,94854054281,18152024243,0.936774,0.816445,1.517199e+02,2024-01-22 02:23:01.592837
1,bitcoin,btc,Bitcoin,41350.000000,811511721498,1,869267288955,9510001552,38753.514527,33775.625586,6.276519e+06,2024-01-22 02:23:01.592837
2,ethereum,eth,Ethereum,2440.320000,293742017002,2,293742017002,4968625807,2287.085286,1993.309181,3.704163e+05,2024-01-22 02:23:01.592837
3,usd-coin,usdc,USDC,0.999842,25778685278,7,25496015780,2620408718,0.937059,0.816694,1.517661e+02,2024-01-22 02:23:01.592837
4,first-digital-usd,fdusd,First Digital USD,1.000000,2615577808,36,2615577808,1412102625,0.937207,0.816823,1.517901e+02,2024-01-22 02:23:01.592837
5,solana,sol,Solana,90.410000,39231211937,5,51434835010,964185119,84.732896,73.848955,1.372334e+04,2024-01-22 02:23:01.592837
6,dogecoin,doge,Dogecoin,0.083275,11916839217,10,11916851738,888890806,0.078046,0.068021,1.264032e+01,2024-01-22 02:23:01.592837
7,binancecoin,bnb,BNB,317.920000,48885311491,4,48885311491,517250791,297.956888,259.684326,4.825710e+04,2024-01-22 02:23:01.592837
8,ripple,xrp,XRP,0.542096,29493933171,6,54270293527,484986645,0.508056,0.442796,8.228479e+01,2024-01-22 02:23:01.592837
9,arbitrum,arb,Arbitrum,1.790000,2283226806,40,17907661226,409069888,1.677601,1.462113,2.717042e+02,2024-01-22 02:23:01.592837


In [ ]:
host = 'testtech.postgres.database.azure.com'
user = 'testtech'
database = 'postgres'
password = 'Your_password'

## Loading to Azure postgresql database

In [29]:
import psycopg2
from sqlalchemy import create_engine #pip install both packages
def load(dataframe)
    try:

        #Create a dictionary of our credentials
        connection_params = {
        "host": "testtech.postgres.database.azure.com",
        "port": "5432",
        "user": "testtech",
        "password": "Your_password",
        "database": "postgres"
    }

    #Create a SQLalchemy engine for connecting to database
        engine = create_engine(f'postgresql+psycopg2://{connection_params["user"]}:{connection_params["password"]}@{connection_params["host"]}:{connection_params["port"]}/{connection_params["database"]}')
    #Append the dataframe contents to the existing table and it'll create it if it's not there.
        dataframe.to_sql('trending_coins' , engine, if_exists = 'append', index = False)

        print('Database Successfully updated')

    except Exception as e:
        print('An error occurred:', e)

    finally:
        if engine:
            engine.dispose()

Database Successfully updated
